In [74]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection


In [75]:
from langchain_community.document_loaders import TextLoader

In [76]:
loader = [ 
    TextLoader("datatxt/GLGuide1.txt", encoding="ISO-8859-1"),
    TextLoader("datatxt/Commonclauses1.txt", encoding="ISO-8859-1"),
    TextLoader("datatxt/Declarations31.txt", encoding="ISO-8859-1")
]


In [77]:
document = []

In [78]:
for loader_instance in loader:
    document.extend(loader_instance.load()) 

In [79]:
document

[Document(metadata={'source': 'datatxt/GLGuide1.txt'}, page_content='Pronto RAD 4GL functions\nArithmetic functions\nAAND()\nDescription\nReturns the arithmetic (binary) AND of two binary numbers\nSyntax\nname=\ndetail=AAND(binary_number,binary_number)\nCategory\nArithmetic\nRuntime version\n3.0 and above\nset ws-num = aand(13,5)\n // // Result: ws-num = 5 // // 1101 (13) // 0101 (5) // ---------- // 0101 (5) (ie. bit positions 2 and 4 are set (1) in both bytes) set ws-num = aand(9,2)\n // // Result: ws-num = 0 // // 1001 (9) // 0010 (2) // ---------- // 0000 (0) That is, no bit positions are set (1) in both bytes.)\nABS()\nDescription\nReturns the absolute (positive) value of a number\nSyntax\nABS(number)\nCategory\nArithmetic\nRuntime version\n3.1 and above\nset gl-trans-amount = abs(ws-invoice-amt)\n // // Result: If ws-invoice-amt is -99.00 then // gl-trans-amount is 99.00 \nANOT()\nDescription\nReturns the arithmetic (binary) NOT of a binary number\nSyntax\nANOT(binary_number)\nCa

In [80]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=40)
chunks=text_splitter.split_documents(document)
for chunk in chunks:
    print(chunk)
    print("----------")

page_content='Pronto RAD 4GL functions
Arithmetic functions
AAND()
Description
Returns the arithmetic (binary) AND of two binary numbers
Syntax
name=
detail=AAND(binary_number,binary_number)
Category
Arithmetic' metadata={'source': 'datatxt/GLGuide1.txt'}
----------
page_content='Category
Arithmetic
Runtime version
3.0 and above
set ws-num = aand(13,5)' metadata={'source': 'datatxt/GLGuide1.txt'}
----------
page_content='3.0 and above
set ws-num = aand(13,5)
 // // Result: ws-num = 5 // // 1101 (13) // 0101 (5) // ---------- // 0101 (5) (ie. bit positions 2 and 4 are set (1) in both bytes) set ws-num = aand(9,2)' metadata={'source': 'datatxt/GLGuide1.txt'}
----------
page_content='// // Result: ws-num = 0 // // 1001 (9) // 0010 (2) // ---------- // 0000 (0) That is, no bit positions are set (1) in both bytes.)
ABS()
Description' metadata={'source': 'datatxt/GLGuide1.txt'}
----------
page_content='ABS()
Description
Returns the absolute (positive) value of a number
Syntax
ABS(number)
Cat

In [81]:
from sentence_transformers import SentenceTransformer
embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", trust_remote_code=True)

In [82]:
em  = embeddings.encode('My name is vibhav')
len(em)

384

In [83]:
from pymilvus import connections, db, utility
conn = connections.connect('default', host = '127.0.0.1', port = 19530)

In [84]:
db.list_database()

['default', 'firstdb', 'second']

In [85]:
db.create_database("firstdb")

RPC error: [create_database], <MilvusException: (code=65535, message=database already exist: firstdb)>, <Time:{'RPC start': '2024-12-13 13:59:47.106158', 'RPC error': '2024-12-13 13:59:47.108505'}>


MilvusException: <MilvusException: (code=65535, message=database already exist: firstdb)>

In [86]:
db.list_database()

['default', 'firstdb', 'second']

In [87]:
db.using_database('firstdb')


In [88]:
utility.list_collections()

[]

In [91]:
id_field = FieldSchema(name='id', dtype=DataType.INT64, is_primary = True, auto_id = True)
source_field = FieldSchema(name='source', dtype=DataType.VARCHAR, max_length = 150,is_primary = False)
embedding_field = FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, dim=384)
content_field = FieldSchema(name='content', dtype=DataType.VARCHAR, max_length = 500)

schema  = CollectionSchema(fields=[id_field,source_field,embedding_field,content_field])


In [92]:
collection = Collection(name='Milvus', schema=schema)

In [93]:
utility.list_collections()

['Milvus']

In [94]:
index_params={
    'metric_type': 'L2',
    'index_type': 'HNSW',
    'params': {
        'M':16,
        'efConstruction': 100
    }
}

In [95]:
collection.create_index(field_name='embedding', index_params=index_params)

Status(code=0, message=)

In [96]:
collection.load()

In [98]:
sources =[chunk.metadata['source'] for chunk in chunks]
page_content =[chunk.page_content for chunk in chunks]
content_embedding =[embeddings.encode(chunk.page_content) for chunk in chunks]


In [101]:
sources

['datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',
 'datatxt/GLGuide1.txt',


In [103]:
collection.insert([sources,content_embedding,page_content])

(insert count: 1204, delete count: 0, upsert count: 0, timestamp: 454579733806448643, success count: 1204, err count: 0

In [104]:
db.using_database('firstdb')

In [105]:
collection = Collection(name='Milvus')
collection.load()

In [106]:
collection

<Collection>:
-------------
<name>: Milvus
<description>: 
<schema>: {'auto_id': True, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'source', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 150}}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}, {'name': 'content', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 500}}], 'enable_dynamic_field': False}